In [1]:
import sko
import numpy as np
import pandas
import sys

In [2]:
'''
0 eta 保护渣粘度
1 f 结晶器频率
2 s 结晶器行程
3 Tsol 保护渣冷凝温度
4 Rp 熔池排液速率
5 p 保护渣密度
6 M_L 模具长度
7 Vw 冷却水速率
8 DH 水力学直径
9 M_t 模具厚度
10 Tw 冷却水温度
11 Szl 喷淋区长度
12 SZt 喷淋区温度
13 Szep 喷淋区出口辊距
14 Mep 结晶器出口辊距

'''
#拉坯速度
global Vc
Vc=1.2

#表面积体积比
global v_a
v_a=0.25

#铸坯表面温度
global Tsurf
Tsurf=1250

#剩余壳层区
global remain_shell
remain_shell=50

#连铸机总长度
global Machine_length
Machine_length=45

#宽面尺寸
global wide_face_dimension
wide_face_dimension=1400

#辊距
global roll_picth
roll_pitch=250

#铸造半径
global caster_radius
caster_radius=8

#结晶器出口坯壳厚度
global shell_m_e
shell_m_e=15

#喷淋区出口坯壳厚度
global shell_s_e
shell_s_e=98

#结晶器内热应变
global ther_strain_m
ther_strain_m=0.08

#喷淋区内热应变
global ther_strain_s
ther_strain_s=0.1

#结晶器表面温度
global surf_t_m
surf_t_m=1250

#喷淋区表面温度
global surf_t_s
surf_t_s=1200

#液相不可穿透温度
global liquid_im_t
liquid_im_t=1350

#水压
global Pw
Pw=3

#热流量
global q
q=1.4e+6

#结晶器热导率
global Kmold
Kmold=400

#临界应力
global sigma_c
sigma_c=600

#钢铁固相线温度
global steel_solidus_t
steel_solidus_t=1352

#喷淋区出口温度
global spray_e_t
spray_e_t=1230

W_0=[1,0.0016,44.5,44.5,0.0016,6.25e-6,1.44,44.5,44.5,(1.11/sigma_c)**2,0.0004,0.0012,1,(1.11/sigma_c)**2,0.00156,(1/Machine_length)**2,1]
lb=[1,100,6,1000,1,2000,600,10,0.01,7,25,5,1000,200,100]
ub=[2,200,12,1100,3,3000,800,30,0.03,15,35,6,1100,300,150]

def X0_init(lb,ub):
    from numpy.random import uniform
    if len(lb)!=len(ub):
        print('边界尺寸不匹配')
        return
    X0=uniform(low=lb,high=ub)
    return X0

In [3]:
Loss_fun_list=[]
for i in range(0,17):
    Loss_fun_list.append("loss_"+str(i+1))

def loss_1(X,Vc=Vc):
    eta=X[0]
    C1=eta*Vc
    L1=(C1-2)**2
    return L1

def loss_2(X):
    eta=X[0]
    Tsol=X[3]
    C2=Tsol/np.power(eta,0.0472)
    L2=(C2-1150)**2
    return L2

def loss_3(X,Vc=Vc):
    eta=X[0]
    Tsol=X[3]
    C3=1.952-0.2461*Vc-0.044*eta-0.00107*Tsol
    L3=(C3-0.3)**2
    return L3

def loss_4(X,Vc=Vc):
    eta=X[0]
    f=X[1]
    s=X[2]
    C4=0.74*np.power(2/s,0.3)*(60/f)*np.power(eta*(Vc)**2,-0.5)+0.17
    L4=(C4-0.3)**2
    return L4

def loss_5(X,Vc=Vc):
    f=X[1]
    C5=Vc/f*1000
    L5=(C5)**2
    return L5

def loss_6(X):
    f=X[1]/60
    s=X[2]
    C6=600*np.sqrt(s/(10*f))
    L6=(C6)**2
    return L6

def loss_7(X,Vc=Vc):
    f=X[1]
    s=X[2]
    Va=np.pi*f*s/1000
    Vm=2*Va/np.pi
    C7=Vm/Vc
    L7=(1/C7)**2
    return L7

def loss_8(X):
    Rp=X[4]/1000
    p=X[5]
    C8=(Rp*p/(Vc))*v_a
    L8=(C8-0.3)**2
    return L8

def loss_9(X):
    eta=X[0]
    f=X[1]
    s=X[2]/1000
    Tsol=X[3]
    M_L=X[6]/1000
    C9=np.power(eta,0.5)*np.power(s,-0.25)*np.power(f,0.25)*np.power(Vc,0.25)*np.sqrt(1/60)
    C9=C9*0.251*(Tsurf/Tsol)*(M_L/Vc)
    L9=(C9-0.3)**2
    return L9 

def loss_10(X):
    M_L=X[6]
    Mzp=X[14]
    C10=7.781+0.00372*wide_face_dimension\
        +0.00186*M_L+0.00175*Mzp\
        +0.0109*caster_radius\
        -0.044*shell_m_e\
        +0.671*ther_strain_m\
        -0.00285*surf_t_m\
        -0.00381*liquid_im_t
    L10=(C10)**2
    return L10

def loss_11(X):
    M_t=X[9]/1000
    Vw=X[7]
    DH=X[8]
    Tw=X[10]
    C11=(M_t/Kmold+1)/(1.26*np.power(Vw/8.46e-5,0.8)*np.power(DH/0.305,-0.2))
    C11=q*C11+Tw
    L11=(C11-200)**2
    return L11

def loss_12(X):
    M_t=X[9]/1000
    Vw=X[7]
    DH=X[8]
    Tw=X[10]
    C12=1/(1.26*np.power(Vw/8.46e-5,0.8)*np.power(DH/0.305,-0.2))
    C12=q*C12+Tw
    L12=(C12-100)**2
    return L12

def loss_13(X):
    M_L=X[6]/1000
    C13=M_L/np.power(wide_face_dimension/1000,0.3)
    L13=(1/C13)**2
    return L13

def loss_14(X):
    Szl=X[11]
    Szep=X[13]
    C14=8.45+0.0077*wide_face_dimension\
        +0.00392*Szl+0.00317*Szep\
        +0.0267*caster_radius\
        -0.0418*shell_s_e\
        +0.556*ther_strain_s\
        -0.00389*surf_t_s\
        -0.00392*liquid_im_t
    L14=(C14)**2
    return L14

def loss_15(X):
    Szt=X[12]
    C15=445.8-2.8*(shell_s_e/1000)+0.52*steel_solidus_t-0.98*Szt
    L15=(C15)**2
    return L15

def loss_16(X):
    Szl=X[11]
    M_L=X[6]/1000
    C16=2.247+0.00124*Vc*np.power(remain_shell,2)+Szl+M_L
    L16=(C16)**2
    return L16

def loss_17(X):
    Vw=X[7]
    C17=Vw/(40.29*np.power(Pw,-0.282))
    L17=(1/C17)**2
    return L17

def loss_sum(X,W=W_0,loss_list=Loss_fun_list):
    L=0
    for i in range(len(loss_list)):
        func=loss_list[i]
        L+=(globals()[func](X))*W[i]
    return L

In [4]:
from sko.SA import SA
X_0=X0_init(lb,ub)
opt=SA(func=loss_sum,x0=X_0,lb=lb,ub=ub,max_stay_counter=350,T_max=3500)
best_x,best_y=opt.run()
for i in range(len(Loss_fun_list)):
    func=Loss_fun_list[i]
    print(i+1,':')
    print((globals()[func](best_x))*W_0[i])

1 :
0.6400000000000001
2 :
4.0
3 :
0.8192032767999983
4 :
0.002609327164447164
5 :
0.057600000000000005
6 :
0.5357733672345155
7 :
0.20570758436858685
8 :
0.6056944444444448
9 :
0.025018697343719754
10 :
0.00010334217755097636
11 :
6.538618389986857
12 :
0.9310815270039071
13 :
1.912039444913217
14 :
0.00013438263018331108
15 :
7.768026089241629
16 :
0.06837643901234568
17 :
2.5148018073641873
